<a name="top"> <h1>2. FALSOS REEMPLAZOS</h1> <a>


<p>CREATION OF BBDD<br />
<strong>Price to Market Motor</strong><br />



<p style="text-align:right">Miguel D&iacute;az-Mauriño (<em>miguel.diaz@allianz.es</em>)</p>

# Setup notebooks

### General libraries

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import pyreadstat as prs
import logging
import os
import pathlib

from io import BytesIO
from azure.identity import AzureCliCredential
from azure.storage.filedatalake import DataLakeServiceClient, DataLakeDirectoryClient, DataLakeFileClient
from os.path import join

pd.set_option("display.max_columns", None)
pd.options.display.float_format = '{:,.2f}'.format

### Logging

In [2]:
import logging

logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

### Load enviroment variables and adding to PYTHONPATH

In [3]:
from dotenv import load_dotenv
load_dotenv('.env')

code_root = os.environ['CODE_ROOT']
cfg_path = os.environ['CFG_PATH']
data_root = os.environ['DATA_ROOT']

sys.path.insert(0, code_root)

### Specific libraries

In [4]:
from src.data_dict import ratios 

### Configuration conecction to DB

In [5]:
class AzureDataLakeGen2:
    """
    This class encapsulates Data Backup Export methods and variables.
    Can be imported to read and write CSV files and SAV files to Azure Data Lake.
    """

    def __init__(self, container_name: str = "advanced-analytics", storage_account_name: str = "prodazesqubi"):
        """
        Initializes an instance of AzureDataLakeGen2 with the provided container name and storage account name.

        :param container_name: The name of the container in Azure Data Lake.
        :param storage_account_name: The name of the storage account in Azure Data Lake.
        """
        self.account_url = f"https://{storage_account_name}.dfs.core.windows.net/"
        self.cli_credential = AzureCliCredential()
        self.datalake_service_client = DataLakeServiceClient(account_url=self.account_url, credential=self.cli_credential,
                                                             proxies={ "https": "http://gdp-squid-proxy.dsp1.we1.azure.aztec.cloud.allianz"})
        self.file_system_client = self.datalake_service_client.get_file_system_client(file_system=container_name)
        
        # Logging configuration
        self.logger = logging.getLogger("azure.core.pipeline.policies.http_logging_policy")
        self.logger.setLevel(logging.WARNING)

    @staticmethod
    def _get_file_name(file_path: str) -> str:
        """
        Extracts the file name from a given file path.

        :param file_path: The file path to extract the file name from.
        :return: The file name.
        """
        path = pathlib.Path(file_path)
        return str(pathlib.Path(*path.parts[1:]))

    def _create_file_client(self, file_path: str) -> DataLakeFileClient:
        """
        Creates a new DataLakeFileClient for the given file path.

        :param file_path: The file path to create the DataLakeFileClient for.
        :return: The DataLakeFileClient.
        """
        path_str_list = file_path.split("/")
        directory_client = self.file_system_client.get_directory_client(f"{path_str_list[0]}")
        return directory_client.create_file(self._get_file_name(file_path))

    def _get_file_client(self, file_path: str) -> DataLakeFileClient:
        """
        Gets the DataLakeFileClient for the given file path.

        :param file_path: The file path to get the DataLakeFileClient for.
        :return: The DataLakeFileClient.
        """
        path_str_list = file_path.split("/")
        directory_client = self.file_system_client.get_directory_client(f"{path_str_list[0]}")
        return directory_client.get_file_client(self._get_file_name(file_path))

    def write_file_to_blob(self, file_path: str, data: bytes) -> None:
        """
        Writes the provided data to the specified file in Azure Data Lake.

        :param file_path: The file path to write the data to.
        :param data: The data to write.
        """
        file_write_client = self._create_file_client(file_path.replace("\\", "/"))
        file_write_client.upload_data(data, len(data), overwrite=True)
        self.logger.info(f"Wrote data to file {file_path}")

    def write_model_to_blob(self, file_path: str, data=None):
        """
        Uploads the given binary data to the specified file path in the Data Lake file system.

        :param file_path: str - The file path to upload the data to.
        :param data: bytes - The binary data to upload.
        """
        file_path = file_path.replace("\\", "/")
        with self._create_file_client(file_path) as file_write_client:
            file_write_client.upload_data(data, overwrite=True)

    def read_raw_file(self, file_path: str):
        """
        Downloads the contents of the file at the given file path in the Data Lake file system as bytes.

        :param file_path: str - The file path to download.
        :return: BytesIO - A BytesIO object containing the downloaded file contents.
        """
        file_path = file_path.replace("\\", "/")
        with self._get_file_client(file_path) as file_client:
            download = file_client.download_file()
            download_bytes = download.readall()
        return BytesIO(download_bytes)

In [6]:
!az login

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "6e06e42d-6925-47c6-b9e7-9581c7ca302a",
    "id": "7f616aeb-a689-468d-850b-35d57d72ae8b",
    "isDefault": true,
    "managedByTenants": [],
    "name": "GDP-21-westeurope-prod",
    "state": "Enabled",
    "tenantId": "6e06e42d-6925-47c6-b9e7-9581c7ca302a",
    "user": {
      "name": "miguel.diaz@allianz.es",
      "type": "user"
    }
  }
]


In [7]:
dl = AzureDataLakeGen2(container_name='ramoaut', storage_account_name='prodazesbisas')

In [8]:
byte_io = dl.read_raw_file('CARTERA_AUTOS/LATEST/CARTERA_AUTOS.parquet')

# https://prodazesbisas.blob.core.windows.net/cartera/CAR_AUTOS/LATEST/CAR_AUTOS.parquet

2024-04-25 08:35:32,431 - AzureCliCredential.get_token succeeded


In [9]:
df = pd.read_parquet(byte_io)

In [10]:
df.head(10)

,FECHA,CIA,POLIZA,APLICA,AGRUPACM,RAMO,DC,SUCURSAL,UC,AGENTE,AGENTE_OLD,TOMADOR,PRIANUALAC,PRIANUALAA,FALTA,FALTA1,FEFECTO,FTERM,FMODIF,FCART,ICART_DIA,ICART_MES,FANUL,FIANUL,MOTANUL,FRPOLEXP,USO,FP,ICOAS,CIACOA_A,PCT_COASEGURO_CEDIDO,PCT_COASEGURO,ANUL_ANT_81,CARTE_AN_68,CARTE_AN_69,CARTE_AN_87,CARTE_AN_88,NUEPR_AN_77,NUEPR_AN_82,NUEPR_AN_83,NUEPR_AN_84,NUEPR_AN_85,NUEPR_NR_89,NUEPR_RE_90,PACRAS,POLIZAS_45,PRIMAN,PRIMAS_46,PRIMAB,IREAS,CIAREA_A,PCT_REASEGURO,PML,CAPTOTAL,BRUTCOAS_47,POLIZAR,APLICAR,POLIZAN,APLICAN,POLIZA31,APLICA31,RAMO31,POLIZAO,APLICAO,CLCOBRO,CODEVID,CODTRAT,USUARIOD,NUM_PROYECTO,FLOTA,REEMPLAZOSI,REEMPLAZOSF,NCUMULO,USERALTA,EMPLEADO_BPE,COLABORADOR,FACULTATIVO,COMESP,CODPRODUCTO,PCT_CLIENTE_AP,CONCURSO,POLANUL_NP,POLANUL_CAR,MONEDA,PCT_DTO_VENTA_CRUZADA,USERMODI,SINGULAR,DST_BANCO,DST_OFICINA_BANC,DTO_COMERCIAL,COMISION_ESP,DTO_DEFACUM,HORA_EMISION,SUBPRODUCTO,SEGMEN,PAQUETE_TREN,MODALIDAD,PCT_COMISION_BASE,DISPOSITIVO,ENTORNO,PCT_DTO_FLOTA,IND_CBB,IND_LPS,PCT_DTO_TRASPASO,FEMIS,FVTO_AC,DIAS_ANUALIDAD_ANT,DIAS_ANUALIDAD_ACT,NIF_TIPO,CODPOST,agrupacm31,polizas_75,NUEPR_AA_79,POLIZAF,APLICAF,IND_NP,IND_NP_MTD,IND_DIF,IND_REN,PRIMAN_CAR,PRIMAN_NP,PRIMAN_REN,POLIZAS_REN,PCT_DTO_POLIZA,DTO,BM_BASICO,BNS_ANTIGUEDAD,BNS_ANYOS_SIN_SINIESTROS,BNS_CIAORIGEN,BNS_RECIBO_ANT,BNS_RECIBO_PROX,BNS_SEGURO_PREVIO,BNS_STROS_12MESES,BNS_TIPO_BONIFICACION,BNS_TIPO_CONSULTA,BNS_POLORIGEN,BNS_MATRICULA,BNS_DOMICILIA_BANCO,CORREC_BM_ANTEC_AGENTE,CORREC_BM_ANTEC_TIREA,CORREC_BM_5ANYOS,CORREC_BM_PROPIEDAD,CORREC_BM_EMPRESA,CORREC_BM_DOMICILIA_BANCO,CORREC_BM_COND_VCIAL,CORREC_BM_FAMILIAR,CORREC_BM_NUEVO,CORREC_BM_2COND,CORREC_PRIMA_ANTEC,CORREC_BM_FINAL,BM_INICIAL,BM_ANTERIOR,BM_ACTUAL,POR_BONIFICACION_AA,POR_BONIFICACION_AC,PROPIEDAD_ANT_VEH,VALOR_STROS_AA,VALOR_STROS_AC,SEX,ECIVIL_COND,PROF,SEX2,CAUT1824D,CAUT1824H,CAUT1824M,CAUT2534M,CAUTU19D,CAUTU19H,ZRU,HABITANTES,ZRIESGO,CTVEHICIAL,TURFAMI2,ASISTENCIA,DEFENSA,KMANU,AUTOS_KM_72,AUTOS_KM_76,AUTOS_KM_80,INTERNAC,CNBREPARTO,MIP,DANPRE,PROP_JURID,TOM_JURID,COND_TOM,COND_PROP,TALLER,CONYUGE,GARAJE_LABORAL,GARAJE,R3RADIO,R3RADIO50,PERTOTAL,TOMFD2,CAPITAL_ACC,CODVH,CODVH11,MATRICULA,CILIND,POTENCIA,CLASEVH,PESOVH,VALORVH,VALORAC,POLSVA,MARCA,MODELO,ICEAN,B7_DISENY,OCUPANTE,ITV,FACTOR1,FACTOR2,FACTOR3,COMBUST,BASICO,ROBO,DANOS,FRANQUI,CAMPANYA,CAMPANYA_DTO,DETECTOR,REPATRIACION,TIPOCOMISION,TIPOCOMISION_DTO,COBERTURA,PROTECCIONBONUS,ESP,TOPTEN,TIREA_MOD,PACK_SEG,PAQUETE,PAQUETE_REPORTING,SATELITE,POLIZA_FAM,APLICA_FAM,AEROPUERTO,PLAZAS,CORREC_NB_VEH,TABLA_TARIFA,GRUA,COND25,TIPO_REEMPLAZO,CODPROV,OBSERVACION,CHASIS,EPAC,ASIGNA_TT,CARNET_CE,VALORRE,SAT_FDESINST_AUX,SAT_COD_PROVEED,SAT_LT_PROVEED,TABLA_TARIFA2,IMPORTE_FRANQUICIA,RECA2C,AUT25,VALOR15000,NOPERATIVA,IVA_INC,NPROPIETARIO,ID_CLIENTE,FNACI8,FCARN8,FNACI82,FCARN82,FMATRI,FADQUI,SAT_FCERT_INST,SAT_FREC_INST,SAT_FINST,SAT_FCERT_DESINST,FMATRI_calc,ANTIG_VEHICULO,BM_ACTUAL_ACTUARIAL,BM_ANTERIOR_ACTUARIAL,PQ_BA,PQ_DA,PQ_RO,PQ_PT,PQ_DACFR,EXP_BA,EXP_DA,EXP_RO,EXP_DASFR,EXP_DACFR,EDADREN,EDADAUT,ANTCARN,ANTGAUT,TIPO_CONDUCTOR_AUTORIZADO,IND_CARNET
0,2024-03-31,2.00,"54,680,741.00",0.00,"1,020.00","1,289.00",21.00,550.00,C,"5,500,713.00","5,500,713.00","64,679,296.00",343.43,0.00,2023-12-05,2023-12-31,2023-12-31,2025-01-01,2023-12-05,2025-01-01,1.00,1.00,NaT,NaT,None,0.25,T 10,1.00,N,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,I,1.00,343.43,343.43,343.43,N,NaN,0.00,0.00,0.00,343.43,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"54,680,741.00",0.00,06,0.00,None,None,0.00,N,0.00,0.00,0.00,PA184380,0.00,0.00,N,N,0.00,0.12,N,0.00,0.00,907.00,0.00,None,None,"2,100.00","1,657.00",0.00,0.00,0.00,2024-04-21 10:31:41,0.00,0.00,3.00,0.00,0.12,desktop,EP,0.00,N,N,0.00,2023-12-05,2025-01-01,NaN,91.00,N,36212,NaN,1.00,0.00,"54,680,741.00",0.00,0.00,0.00,0.00,0.00,343.43,0.00,0.00,0.00,0.00,0.00,13.00,0.00,5.00,C517,N,S,S,0.00,ANT,CT,56153,None,None,-11.00,-100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12.00,1.00,0.00,1.

In [11]:
df.POLIZA = df.POLIZA.astype('int')
df.RAMO = df.RAMO.astype('int')
df.TOMADOR = df.TOMADOR.astype('int')

In [12]:
df.head(2)

,FECHA,CIA,POLIZA,APLICA,AGRUPACM,RAMO,DC,SUCURSAL,UC,AGENTE,AGENTE_OLD,TOMADOR,PRIANUALAC,PRIANUALAA,FALTA,FALTA1,FEFECTO,FTERM,FMODIF,FCART,ICART_DIA,ICART_MES,FANUL,FIANUL,MOTANUL,FRPOLEXP,USO,FP,ICOAS,CIACOA_A,PCT_COASEGURO_CEDIDO,PCT_COASEGURO,ANUL_ANT_81,CARTE_AN_68,CARTE_AN_69,CARTE_AN_87,CARTE_AN_88,NUEPR_AN_77,NUEPR_AN_82,NUEPR_AN_83,NUEPR_AN_84,NUEPR_AN_85,NUEPR_NR_89,NUEPR_RE_90,PACRAS,POLIZAS_45,PRIMAN,PRIMAS_46,PRIMAB,IREAS,CIAREA_A,PCT_REASEGURO,PML,CAPTOTAL,BRUTCOAS_47,POLIZAR,APLICAR,POLIZAN,APLICAN,POLIZA31,APLICA31,RAMO31,POLIZAO,APLICAO,CLCOBRO,CODEVID,CODTRAT,USUARIOD,NUM_PROYECTO,FLOTA,REEMPLAZOSI,REEMPLAZOSF,NCUMULO,USERALTA,EMPLEADO_BPE,COLABORADOR,FACULTATIVO,COMESP,CODPRODUCTO,PCT_CLIENTE_AP,CONCURSO,POLANUL_NP,POLANUL_CAR,MONEDA,PCT_DTO_VENTA_CRUZADA,USERMODI,SINGULAR,DST_BANCO,DST_OFICINA_BANC,DTO_COMERCIAL,COMISION_ESP,DTO_DEFACUM,HORA_EMISION,SUBPRODUCTO,SEGMEN,PAQUETE_TREN,MODALIDAD,PCT_COMISION_BASE,DISPOSITIVO,ENTORNO,PCT_DTO_FLOTA,IND_CBB,IND_LPS,PCT_DTO_TRASPASO,FEMIS,FVTO_AC,DIAS_ANUALIDAD_ANT,DIAS_ANUALIDAD_ACT,NIF_TIPO,CODPOST,agrupacm31,polizas_75,NUEPR_AA_79,POLIZAF,APLICAF,IND_NP,IND_NP_MTD,IND_DIF,IND_REN,PRIMAN_CAR,PRIMAN_NP,PRIMAN_REN,POLIZAS_REN,PCT_DTO_POLIZA,DTO,BM_BASICO,BNS_ANTIGUEDAD,BNS_ANYOS_SIN_SINIESTROS,BNS_CIAORIGEN,BNS_RECIBO_ANT,BNS_RECIBO_PROX,BNS_SEGURO_PREVIO,BNS_STROS_12MESES,BNS_TIPO_BONIFICACION,BNS_TIPO_CONSULTA,BNS_POLORIGEN,BNS_MATRICULA,BNS_DOMICILIA_BANCO,CORREC_BM_ANTEC_AGENTE,CORREC_BM_ANTEC_TIREA,CORREC_BM_5ANYOS,CORREC_BM_PROPIEDAD,CORREC_BM_EMPRESA,CORREC_BM_DOMICILIA_BANCO,CORREC_BM_COND_VCIAL,CORREC_BM_FAMILIAR,CORREC_BM_NUEVO,CORREC_BM_2COND,CORREC_PRIMA_ANTEC,CORREC_BM_FINAL,BM_INICIAL,BM_ANTERIOR,BM_ACTUAL,POR_BONIFICACION_AA,POR_BONIFICACION_AC,PROPIEDAD_ANT_VEH,VALOR_STROS_AA,VALOR_STROS_AC,SEX,ECIVIL_COND,PROF,SEX2,CAUT1824D,CAUT1824H,CAUT1824M,CAUT2534M,CAUTU19D,CAUTU19H,ZRU,HABITANTES,ZRIESGO,CTVEHICIAL,TURFAMI2,ASISTENCIA,DEFENSA,KMANU,AUTOS_KM_72,AUTOS_KM_76,AUTOS_KM_80,INTERNAC,CNBREPARTO,MIP,DANPRE,PROP_JURID,TOM_JURID,COND_TOM,COND_PROP,TALLER,CONYUGE,GARAJE_LABORAL,GARAJE,R3RADIO,R3RADIO50,PERTOTAL,TOMFD2,CAPITAL_ACC,CODVH,CODVH11,MATRICULA,CILIND,POTENCIA,CLASEVH,PESOVH,VALORVH,VALORAC,POLSVA,MARCA,MODELO,ICEAN,B7_DISENY,OCUPANTE,ITV,FACTOR1,FACTOR2,FACTOR3,COMBUST,BASICO,ROBO,DANOS,FRANQUI,CAMPANYA,CAMPANYA_DTO,DETECTOR,REPATRIACION,TIPOCOMISION,TIPOCOMISION_DTO,COBERTURA,PROTECCIONBONUS,ESP,TOPTEN,TIREA_MOD,PACK_SEG,PAQUETE,PAQUETE_REPORTING,SATELITE,POLIZA_FAM,APLICA_FAM,AEROPUERTO,PLAZAS,CORREC_NB_VEH,TABLA_TARIFA,GRUA,COND25,TIPO_REEMPLAZO,CODPROV,OBSERVACION,CHASIS,EPAC,ASIGNA_TT,CARNET_CE,VALORRE,SAT_FDESINST_AUX,SAT_COD_PROVEED,SAT_LT_PROVEED,TABLA_TARIFA2,IMPORTE_FRANQUICIA,RECA2C,AUT25,VALOR15000,NOPERATIVA,IVA_INC,NPROPIETARIO,ID_CLIENTE,FNACI8,FCARN8,FNACI82,FCARN82,FMATRI,FADQUI,SAT_FCERT_INST,SAT_FREC_INST,SAT_FINST,SAT_FCERT_DESINST,FMATRI_calc,ANTIG_VEHICULO,BM_ACTUAL_ACTUARIAL,BM_ANTERIOR_ACTUARIAL,PQ_BA,PQ_DA,PQ_RO,PQ_PT,PQ_DACFR,EXP_BA,EXP_DA,EXP_RO,EXP_DASFR,EXP_DACFR,EDADREN,EDADAUT,ANTCARN,ANTGAUT,TIPO_CONDUCTOR_AUTORIZADO,IND_CARNET
0,2024-03-31,2.00,54680741,0.00,"1,020.00",1289,21.00,550.00,C,"5,500,713.00","5,500,713.00",64679296,343.43,0.00,2023-12-05,2023-12-31,2023-12-31,2025-01-01,2023-12-05,2025-01-01,1.00,1.00,NaT,NaT,None,0.25,T 10,1.00,N,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,I,1.00,343.43,343.43,343.43,N,NaN,0.00,0.00,0.00,343.43,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"54,680,741.00",0.00,06,0.00,None,None,0.00,N,0.00,0.00,0.00,PA184380,0.00,0.00,N,N,0.00,0.12,N,0.00,0.00,907.00,0.00,None,None,"2,100.00","1,657.00",0.00,0.00,0.00,2024-04-21 10:31:41,0.00,0.00,3.00,0.00,0.12,desktop,EP,0.00,N,N,0.00,2023-12-05,2025-01-01,NaN,91.00,N,36212,NaN,1.00,0.00,"54,680,741.00",0.00,0.00,0.00,0.00,0.00,343.43,0.00,0.00,0.00,0.00,0.00,13.00,0.00,5.00,C517,N,S,S,0.00,ANT,CT,56153,None,None,-11.00,-100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12.00,1.00,0.00,1.00,0.00,0.00,None,0.

## Seleccionamos las variables que necesitamos

In [13]:
path_ft_select_file = r"C:\Users\e107829\Desktop\workplace\prce-mkt-mtr-Falsos-Reemplazos\docs\DD_FR_Features.txt"
with open(path_ft_select_file, 'r') as file:
    features = file.read().splitlines()

len(features)

28

In [14]:
df_features = df.loc[:, features]

df_features.head(2)

,POLIZA,TOMADOR,MODALIDAD,SEX,ASISTENCIA,COND_TOM,COND_PROP,MATRICULA,CILIND,POTENCIA,CLASEVH,VALORVH,MARCA,MODELO,BASICO,ROBO,DANOS,FRANQUI,PAQUETE,PLAZAS,GRUA,NPROPIETARIO,ID_CLIENTE,FNACI8,FCARN8,FMATRI,ANTIG_VEHICULO,BNS_TIPO_CONSULTA
0,54680741,64679296,0.00,Mujer,S,S,N,2078BDH,"1,896.00",90.00,T,"14,975.00",SEAT,CORDOBA SPORT 1.9 TDI 90C,None,None,None,N,2,5.00,None,"84,442,791.00","64,679,296.00",1968-10-08,2003-04-01,NaT,23.00,CT
1,54680742,79414710,0.00,Mujer,S,S,S,5556JPT,"1,248.00",75.00,T,"17,315.00",OPEL,CORSA 1.3 CDTI 75 S&S SEL,None,None,None,N,2,5.00,None,"79,414,710.00","79,414,710.00",1963-09-09,1989-02-27,2016-06-01,7.00,FA


## Leemos nuestro Parquet para incluir dichas variables

In [15]:
file_path = r"C:\Users\e107829\Desktop\workplace\prce-mkt-mtr-Falsos-Reemplazos\docs"

df_fr = pd.read_parquet(os.path.join(file_path, "BBDD_FR.parquet"))

In [16]:
df_fr.head(2)

,POLIZA,APLICA,RAMO,prima_prop,PRIMA_ANT,PAQ_REPORTING,FRENOV,agrupacm,PRIMAN,MOTANUL,FANUL,FIANUL,CLASS_ANUL,Anulada,POLIZAN,APLICAN,CODPOST,POLIZAS_45,AGENTE,DC,SUCURSAL,USO,TOMADOR,TXCANAL,CODMEDP,CATTAG,AGRUPAGENTE,prima_reemplazante,prperiod,SINOCU,N_ANT,N_ACTUAL,Sdadpol_12M,DTO_DEFENSA,PRIMA_DEFENSA_ADPR,CORREDOR_PILOTO,sdad_total3p,sdad_total_tom,anualidad_seguro,Edad_conductor,APTP,rwCoeff_forced,retention,retention_def,Retencion_antes,TechPrice,num_stros_ultaño,num_stros_aga_ultaño,total_stros_ultaño,Valor_Mediador_Turismos,Prob_anul,CODMEDP_bnd,Prueba_Piloto_bnd,Super_rentables_bnd,Corredores_bnd,cltv_decile,Aleatorio,Segmentos_sept,segmento_deficiente_bnd,CAPTOTAL_vida,Edad_tomador_bnd,empleados,POL_VIDA,bm_anualidad,BM_cat,IsBMP,Random,Random_22,DEFENDIBLE,bm_ampliado,Stros2,CS,Provincia,m_anul,Sdadpol_12M_anterior,stros_anterior,defendida_SAS,defendida_ant,FR,priman_new_fr,POLIZAR,PAQUETE_REPORTING,paquete_reporting_reempl,polizar_post,tipo_reemplazo,FRFINAL,GARANTIA_1,GARANTIA_2,GARANTIA_3,GARANTIA_4,GARANTIA_5,GARANTIA_6,GARANTIA_7,GARANTIA_8,Gar_missing
0,23175501,0,1265,608.47,568.66,211.00,2023-01-03,1020,"608,47",None,None,None,None,0,0.00,0.00,"18,006.00",1,28062,29,519,11000,13379484,Agente,"28,062.00",-Ninguno,None,"0,00","549,10","0,00",1,1,-69.33,0,0.00,0,-23.30,-14.83,15.00,77.00,Menor 160%,NaN,0.73,0.92,0.77,374.06,0.00,0.00,0.00,N,Media,Resto,1.00,Resto,Resto,8.00,No,NaN,Resto,None,None,None,None,None,None,None,None,None,None,NaN,None,9,18,NaN,0.00,0.00,N,N,NaN,None,0,211-Todo Riesgo con Franquicia,None,0,None,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
1,23176348,0,1269,229.09,210.18,112.00,2023-01-03,1020,"229,09",None,None,None,None,0,0.00,0.00,"13,360.00",1,6150157,26,615,11200,14236689,Corredor,"6,150,157.00",3-Corredor FIRST,None,"0,00","206,10","0,00",1,1,0.00,0,0.00,0,0.00,0.00,15.00,62.00,Menor 130%,NaN,0.77,0.90,0.81,162.69,0.00,0.00,0.00,N,Media,Resto,1.00,Resto,Resto,5.00,Sí,NaN,Resto,None,None,None,None,None,None,None,None,None,None,NaN,None,9,13,NaN,0.00,0.00,N,S,NaN,None,0,112-Basico,None,0,None,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8


In [17]:
merged_df = pd.merge(df_fr, df_features, on=['POLIZA', 'TOMADOR'], how='inner')

merged_df.groupby(['POLIZA', 'TOMADOR']).size().reset_index(name='count')

,POLIZA,TOMADOR,count
0,23175501,13379484,1
1,23176348,14236689,1
2,23176709,92480610,1
3,23177191,92733813,1
4,23177248,92080298,1
...,...,...,...
1860210,54405353,90040523,1
1860211,54406429,91230143,1
1860212,54406914,84207254,1
1860213,54407057,60615209,1


In [18]:
merged_df.head(5)

,POLIZA,APLICA,RAMO,prima_prop,PRIMA_ANT,PAQ_REPORTING,FRENOV,agrupacm,PRIMAN,MOTANUL,FANUL,FIANUL,CLASS_ANUL,Anulada,POLIZAN,APLICAN,CODPOST,POLIZAS_45,AGENTE,DC,SUCURSAL,USO,TOMADOR,TXCANAL,CODMEDP,CATTAG,AGRUPAGENTE,prima_reemplazante,prperiod,SINOCU,N_ANT,N_ACTUAL,Sdadpol_12M,DTO_DEFENSA,PRIMA_DEFENSA_ADPR,CORREDOR_PILOTO,sdad_total3p,sdad_total_tom,anualidad_seguro,Edad_conductor,APTP,rwCoeff_forced,retention,retention_def,Retencion_antes,TechPrice,num_stros_ultaño,num_stros_aga_ultaño,total_stros_ultaño,Valor_Mediador_Turismos,Prob_anul,CODMEDP_bnd,Prueba_Piloto_bnd,Super_rentables_bnd,Corredores_bnd,cltv_decile,Aleatorio,Segmentos_sept,segmento_deficiente_bnd,CAPTOTAL_vida,Edad_tomador_bnd,empleados,POL_VIDA,bm_anualidad,BM_cat,IsBMP,Random,Random_22,DEFENDIBLE,bm_ampliado,Stros2,CS,Provincia,m_anul,Sdadpol_12M_anterior,stros_anterior,defendida_SAS,defendida_ant,FR,priman_new_fr,POLIZAR,PAQUETE_REPORTING,paquete_reporting_reempl,polizar_post,tipo_reemplazo,FRFINAL,GARANTIA_1,GARANTIA_2,GARANTIA_3,GARANTIA_4,GARANTIA_5,GARANTIA_6,GARANTIA_7,GARANTIA_8,Gar_missing,MODALIDAD,SEX,ASISTENCIA,COND_TOM,COND_PROP,MATRICULA,CILIND,POTENCIA,CLASEVH,VALORVH,MARCA,MODELO,BASICO,ROBO,DANOS,FRANQUI,PAQUETE,PLAZAS,GRUA,NPROPIETARIO,ID_CLIENTE,FNACI8,FCARN8,FMATRI,ANTIG_VEHICULO,BNS_TIPO_CONSULTA
0,23175501,0,1265,608.47,568.66,211.00,2023-01-03,1020,"608,47",None,None,None,None,0,0.00,0.00,"18,006.00",1,28062,29,519,11000,13379484,Agente,"28,062.00",-Ninguno,None,"0,00","549,10","0,00",1,1,-69.33,0,0.00,0,-23.30,-14.83,15.00,77.00,Menor 160%,NaN,0.73,0.92,0.77,374.06,0.00,0.00,0.00,N,Media,Resto,1.00,Resto,Resto,8.00,No,NaN,Resto,None,None,None,None,None,None,None,None,None,None,NaN,None,9,18,NaN,0.00,0.00,N,N,NaN,None,0,211-Todo Riesgo con Franquicia,None,0,None,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,0.00,Hombre,S,S,S,0669GBF,"1,560.00",90.00,T,"19,100.00",PEUGEOT,308 CONFORT 1.6 HDI 90 5P,S,S,S,S,4,5.00,None,"13,379,484.00","13,379,484.00",1946-07-20,1968-02-02,2008-02-01,16.00,CT
1,23176348,0,1269,229.09,210.18,112.00,2023-01-03,1020,"229,09",None,None,None,None,0,0.00,0.00,"13,360.00",1,6150157,26,615,11200,14236689,Corredor,"6,150,157.00",3-Corredor FIRST,None,"0,00","206,10","0,00",1,1,0.00,0,0.00,0,0.00,0.00,15.00,62.00,Menor 130%,NaN,0.77,0.90,0.81,162.69,0.00,0.00,0.00,N,Media,Resto,1.00,Resto,Resto,5.00,Sí,NaN,Resto,None,None,None,None,None,None,None,None,None,None,NaN,None,9,13,NaN,0.00,0.00,N,S,NaN,None,0,112-Basico,None,0,None,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,0.00,Hombre,S,S,S,8362CBW,"1,870.00",80.00,F,"16,626.00",RENAULT,KANGOO COMBI EXPRE 1.9DCI,S,N,N,N,1,5.00,None,"14,236,689.00","14,236,689.00",1961-07-03,1984-04-11,2002-11-01,21.00,CT
2,23176709,0,1269,335.54,319.56,112.00,2023-01-02,1020,"335,54",E-DENUNCIADA: POR EL CONTRATANTE,01/02/2024,04/10/2023,None,0,0.00,0.00,"3,177.00",1,3351901,28,335,11000,92480610,Agente,"3,351,901.00",V-Agente excelente a¦o en curso,None,"0,00","305,68","0,00",1,1,0.00,0,0.00,0,0.00,0.00,15.00,81.00,Menor 250%,NaN,0.73,0.77,0.75,136.35,0.00,0.00,0.00,N,Media,Resto,1.00,SRentables,Resto,8.00,No,NaN,Resto,None,None,None,None,None,None,None,None,None,None,NaN,None,None,3,8.00,0.00,0.00,N,N,NaN,None,0,112-Basico,None,0,None,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,0.00,Hombre,S,S,S,2939GBG,"1,196.00",45.00,T,"8,882.00",OPEL,CORSA 1.2 3P 45CV,S,N,N,N,1,5.00,None,"92,480,610.00","92,480,610.00",1942-04-08,1960-05-10,1993-10-01,16.00,NI
3,23177191,0,1269,332.16,266.06,121.00,2023-01-03,1020,"332,16",None,None,None,None,0,0.00,0.00,"32,800.00",1,5330218,21,470,11000,92733813,Agente,"5,330,218.00",V-Agente excelente a¦o en curso,None,"0,00","292,58","0,00",1,2,399.38,0,0.00,0,191.32,55.40,15.00,66.00,Menor 190%,NaN,0.64,0.68,0.77,140.59,1.00,1.00,2.00,N,Media,Resto,1.00,Resto,Resto,8.00,Sí,NaN,Resto,None,None,None,None,None,None,None,None,None,None,NaN,None,None,32,NaN,0.00,0.00,N,N,NaN,None,0,121-Basico + Robo,None,0,None,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,0.00,Mujer,S,S,S,OU0851S,"1,1

### Modelo Nuevo

In [19]:
TP_New = pd.read_csv(
    r"C:\Users\e107829\Allianz\Price to Market - Turismos - Los documentos\17. Reemplazos y Falsos Reemplazos\Decision_Tree\TP_Turismos_202312_newmodels_PtM.csv",
                 sep=';')
TP_New.rename(columns={'TechPrice': 'TPR_new', 'Actual_price': 'APR_new'}, inplace=True)
TP_New.head(3)

,FECHA,POLIZA,APLICA,APR_new,TPR_new,Expected_Loss_Ultimate,Commercial_price,Value_xs_abs_after_disct,Value_xs_abs_before_disct,TechPrice_xgar[TPL_BodilyInjured],TechPrice_xgar[TPL_MaterialDamage],TechPrice_xgar[OwnDamageWithDeductible],TechPrice_xgar[OwnDamageWithoutDeductible],TechPrice_xgar[AtmosphericPhenomena],TechPrice_xgar[WindScreen],TechPrice_xgar[Theft],TechPrice_xgar[Assistance],TechPrice_xgar[TPL_BodilyInjured_PlateReserve],TechPrice_xgar[TPL_MaterialDamage_PlateReserve],TechPrice_xgar[NonModelled]
0,2023-12-31 00:00:00,128501,0,"377,94965820970373","89,309931764129757",0,"377,94965820970373","288,63972644557396","288,63972644557396",0,0,0,0,0,0,0,0,0,0,0
1,2023-12-31 00:00:00,128502,0,"377,94965820970373","89,309931764129757",0,"377,94965820970373","288,63972644557396","288,63972644557396",0,0,0,0,0,0,0,0,0,0,0
2,2023-12-31 00:00:00,23175501,0,"608,47","338,95619130502109","210,91263378370968","608,47","269,51380869497893","269,51380869497893","116,41275686946412","48,341786715303556","120,32294766310272",0,"1,1065350228188711","15,351118157045239","2,4296072682318055","19,858024846649634",0,0,"15,133414762405195"


In [20]:
selected_variables = ['POLIZA', 'APLICA', 'APR_new', 'TPR_new']
TP_New = TP_New.loc[:, selected_variables]

In [21]:
DF_TP_New = pd.merge(merged_df, TP_New, on=['POLIZA', 'APLICA'], how='left')

print(len(merged_df))
print(len(TP_New))
print(len(DF_TP_New))
print(len(merged_df) - len(DF_TP_New))

1860215
2750232
1860215
0


In [22]:
DF_TP_New.groupby(['FRENOV']).size().reset_index(name='count')

,FRENOV,count
0,2023-01-01,145966
1,2023-01-02,149012
2,2023-01-03,151249
3,2023-01-04,139250
4,2023-01-05,144449
5,2023-01-06,167182
6,2023-01-07,179257
7,2023-01-08,164805
8,2023-01-09,146581
9,2023-01-10,158466


In [23]:
DF_Not_TP_New = pd.merge(merged_df, TP_New, on=['POLIZA', 'APLICA'], how='outer', indicator=True)
DF_Not_TP_New = DF_Not_TP_New[DF_Not_TP_New['_merge'] == 'left_only'].drop(columns='_merge')
len(DF_Not_TP_New)

18857

In [24]:
DF_Not_TP_New.groupby(['FRENOV']).size().reset_index(name='count')

,FRENOV,count
0,2023-01-01,17118
1,2023-01-02,1023
2,2023-01-03,291
3,2023-01-04,112
4,2023-01-05,72
5,2023-01-06,67
6,2023-01-07,73
7,2023-01-08,39
8,2023-01-09,18
9,2023-01-10,20


In [25]:
DF_Not_TP_New.FRFINAL.value_counts()

FRFINAL
0.00    18196
1.00      661
Name: count, dtype: int64

In [26]:
DF_TP_New.FRFINAL.value_counts()

FRFINAL
0.00    1843845
1.00      16370
Name: count, dtype: int64

### Modelo Antiguo

In [27]:
TP_Old = pd.read_csv(
    r"C:\Users\e107829\Allianz\Price to Market - Turismos - Los documentos\17. Reemplazos y Falsos Reemplazos\Decision_Tree\TP_Turismos_202312_AN.csv",
                 sep=';')
TP_Old.rename(columns={'TechPrice': 'TPR_old', 'Actual_price': 'APR_old'}, inplace=True)
TP_Old.head(3)

,FECHA,POLIZA,APLICA,APR_old,TPR_old,Expected_Loss_Ultimate,Commercial_price,Value_xs_abs_after_disct,Value_xs_abs_before_disct
0,2023-12-31 00:00:00,54748867,0,"330,94","516,289639898048","438,80503901089656","330,94","-185,349639898048","-185,349639898048"
1,2023-12-31 00:00:00,42714689,0,"2320,43","1687,7325289688808","1285,5036765694929","2320,43","632,697471031119","632,697471031119"
2,2023-12-31 00:00:00,49395838,0,"625,54","709,46681899506939","583,27799023569708","710,840909090909","-83,926818995069425","1,3740900958396196"


In [28]:
selected_variables1 = ['POLIZA', 'APLICA', 'APR_old', 'TPR_old']
TP_Old = TP_Old.loc[:, selected_variables1]

In [29]:
DF_TP_Old = pd.merge(DF_TP_New, TP_Old, on=['POLIZA', 'APLICA'], how='left')

print(len(DF_TP_New))
print(len(TP_Old))
print(len(DF_TP_Old))
print(len(DF_TP_New) - len(DF_TP_Old))

1860215
2074086
1860215
0


In [30]:
DF_TP_Old.groupby(['FRENOV']).size().reset_index(name='count')

,FRENOV,count
0,2023-01-01,145966
1,2023-01-02,149012
2,2023-01-03,151249
3,2023-01-04,139250
4,2023-01-05,144449
5,2023-01-06,167182
6,2023-01-07,179257
7,2023-01-08,164805
8,2023-01-09,146581
9,2023-01-10,158466


In [31]:
DF_Not_TP_Old = pd.merge(DF_TP_Old, TP_Old, on=['POLIZA', 'APLICA'], how='outer', indicator=True)
DF_Not_TP_Old = DF_Not_TP_Old[DF_Not_TP_Old['_merge'] == 'left_only'].drop(columns='_merge')
len(DF_Not_TP_Old)

489154

In [32]:
DF_TP_Old.FRFINAL.value_counts()

FRFINAL
0.00    1843845
1.00      16370
Name: count, dtype: int64

In [33]:
null_values_count = DF_TP_Old.isnull().sum()
null_values_count

POLIZA                    0
APLICA                    0
RAMO                      0
prima_prop                0
PRIMA_ANT                 0
                      ...  
BNS_TIPO_CONSULTA    141270
APR_new               18857
TPR_new               18857
APR_old              489154
TPR_old              489154
Length: 125, dtype: int64

### Incluimos Variables de Toño

In [34]:
path = r'C:\Users\e107829\Allianz\Price to Market - Turismos - Los documentos\12. Hit Ratio\202311\Conversion\BBDD\Toño'

df_sas_07, _ = prs.read_sas7bdat(os.path.join(path, 'rk_mediador_Roll202307.sas7bdat'))
df_sas_11, _ = prs.read_sas7bdat(os.path.join(path, 'rk_mediador_Roll202311.sas7bdat'))
df_sas_12, _ = prs.read_sas7bdat(os.path.join(path, 'rk_mediador_Roll202312.sas7bdat'))

In [35]:
df_sas_07.columns = [str(col) + '_Med' for col in df_sas_07.columns]
df_sas_11.columns = [str(col) + '_Med' for col in df_sas_11.columns]
df_sas_12.columns = [str(col) + '_Med' for col in df_sas_12.columns]

In [36]:
df_sas_07 = df_sas_07.rename(columns={'tirea_NO_desado_Med': 'tirea_NO_deseado_Med',
                                      'tirea_NO_desado_gr_Med': 'tirea_NO_deseado_gr_Med'})

In [37]:
# Creamos los ratios

df_sas_07 = ratios(df_sas_07)
df_sas_11 = ratios(df_sas_11)
df_sas_12 = ratios(df_sas_12)

In [38]:
## Excluimos aquellas variables de Grupo (GR)
df_sas_07 = df_sas_07.loc[:, ~df_sas_07.columns.str.contains('_gr')]
df_sas_11 = df_sas_11.loc[:, ~df_sas_11.columns.str.contains('_gr')]
df_sas_12 = df_sas_12.loc[:, ~df_sas_12.columns.str.contains('_gr')]

In [39]:
len(df_sas_07.columns)

85

In [40]:
df_sas_07.Mediador_Principal_Med = df_sas_07.Mediador_Principal_Med.fillna(0).astype(int)
df_sas_07.MEDIADOR_Med = df_sas_07.MEDIADOR_Med.fillna(0).astype(int)

In [41]:
nan_counts = df_sas_07.isna().sum()
# Convierte la serie en un DataFrame
nan_counts_df = pd.DataFrame(nan_counts, columns=['Number of NaN'])
# Calcula el porcentaje y crea una nueva columna
nan_counts_df['Percentage'] = nan_counts_df['Number of NaN'] / len(df_sas_07) * 100
nan_counts_df['Percentage'] = nan_counts_df['Percentage'].round()

nan_counts_df.sort_values(by='Number of NaN', ascending=False).head(20)

,Number of NaN,Percentage
Ratio_Canc_NB_Med,13708,100.00
Canc_NB_Med,13708,100.00
desv_Canc_NB_Med,13708,100.00
Dcto_poli_VCTRA_Med,13635,99.00
Dcto_coti_VCTRA_Med,13539,99.00
Ratio_Solicitud_Med,13468,98.00
solicitud_Med,13468,98.00
desv_Solicitud_Med,13468,98.00
desv_Reemplazo_Med,12661,92.00
Ratio_Reemplazo_Med,12661,92.00


In [42]:
nan_counts = df_sas_11.isna().sum()
# Convierte la serie en un DataFrame
nan_counts_df = pd.DataFrame(nan_counts, columns=['Number of NaN'])
# Calcula el porcentaje y crea una nueva columna
nan_counts_df['Percentage'] = nan_counts_df['Number of NaN'] / len(df_sas_11) * 100
nan_counts_df['Percentage'] = nan_counts_df['Percentage'].round()

nan_counts_df.sort_values(by='Number of NaN', ascending=False).head(20)

,Number of NaN,Percentage
desv_Canc_NB_Med,13423,100.00
Canc_NB_Med,13423,100.00
Ratio_Canc_NB_Med,13423,100.00
Dcto_coti_VCTRA_Med,13419,100.00
Dcto_poli_VCTRA_Med,13414,100.00
solicitud_Med,13394,100.00
desv_Solicitud_Med,13394,100.00
Ratio_Solicitud_Med,13394,100.00
Ratio_Reemplazo_Med,12880,96.00
Reemplazo_Med,12880,96.00


In [43]:
nan_counts = df_sas_12.isna().sum()
# Convierte la serie en un DataFrame
nan_counts_df = pd.DataFrame(nan_counts, columns=['Number of NaN'])
# Calcula el porcentaje y crea una nueva columna
nan_counts_df['Percentage'] = nan_counts_df['Number of NaN'] / len(df_sas_12) * 100
nan_counts_df['Percentage'] = nan_counts_df['Percentage'].round()

nan_counts_df.sort_values(by='Number of NaN', ascending=False).head(20)

,Number of NaN,Percentage
Canc_NB_Med,13317,100.00
Ratio_Canc_NB_Med,13317,100.00
desv_Canc_NB_Med,13317,100.00
Dcto_coti_VCTRA_Med,13313,100.00
Dcto_poli_VCTRA_Med,13310,100.00
Ratio_Solicitud_Med,13288,100.00
desv_Solicitud_Med,13288,100.00
solicitud_Med,13288,100.00
desv_Poli_TRA_Med,12786,96.00
Dcto_poli_TRA_Med,12786,96.00


In [44]:
condition1 = ['2023-01-03', '2023-01-01', '2023-01-02', '2023-01-04', '2023-01-05', '2023-01-06', '2023-01-07', '2023-01-08']
condition2 = ['2023-01-09', '2023-01-10', '2023-01-11']
condition3 = ['2023-01-12']

BBDD_1 = DF_TP_Old[DF_TP_Old['FRENOV'].isin(condition1)]
if not BBDD_1.empty:
    BBDD_1 = pd.merge(BBDD_1, df_sas_07, left_on='AGENTE', right_on='MEDIADOR_Med', how='inner')

BBDD_2 = DF_TP_Old[DF_TP_Old['FRENOV'].isin(condition2)]
if not BBDD_2.empty:
    BBDD_2 = pd.merge(BBDD_2, df_sas_11, left_on='AGENTE', right_on='MEDIADOR_Med', how='inner')

BBDD_3 = DF_TP_Old[DF_TP_Old['FRENOV'].isin(condition3)]
if not BBDD_3.empty:
    BBDD_3 = pd.merge(BBDD_3, df_sas_12, left_on='AGENTE', right_on='MEDIADOR_Med', how='inner')

# Combine all the merged dataframes into one
BBDD_VF = pd.concat([BBDD_1, BBDD_2, BBDD_3], ignore_index=True)

C:\Users\e107829\AppData\Local\Temp\ipykernel_21508\2520920319.py:5: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  BBDD_1 = DF_TP_Old[DF_TP_Old['FRENOV'].isin(condition1)]
C:\Users\e107829\AppData\Local\Temp\ipykernel_21508\2520920319.py:9: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  BBDD_2 = DF_TP_Old[DF_TP_Old['FRENOV'].isin(condition2)]
C:\Users\e107829\AppData\Local\Temp\ipykernel_21508\2520920319.py:13: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matchi

In [45]:
print(len(BBDD_VF))
print(len(DF_TP_Old))

1858326
1860215


In [46]:
print(len(DF_TP_Old.columns))
print(len(BBDD_VF.columns))

125
210


In [47]:
BBDD_VF.head(2)

,POLIZA,APLICA,RAMO,prima_prop,PRIMA_ANT,PAQ_REPORTING,FRENOV,agrupacm,PRIMAN,MOTANUL,FANUL,FIANUL,CLASS_ANUL,Anulada,POLIZAN,APLICAN,CODPOST,POLIZAS_45,AGENTE,DC,SUCURSAL,USO,TOMADOR,TXCANAL,CODMEDP,CATTAG,AGRUPAGENTE,prima_reemplazante,prperiod,SINOCU,N_ANT,N_ACTUAL,Sdadpol_12M,DTO_DEFENSA,PRIMA_DEFENSA_ADPR,CORREDOR_PILOTO,sdad_total3p,sdad_total_tom,anualidad_seguro,Edad_conductor,APTP,rwCoeff_forced,retention,retention_def,Retencion_antes,TechPrice,num_stros_ultaño,num_stros_aga_ultaño,total_stros_ultaño,Valor_Mediador_Turismos,Prob_anul,CODMEDP_bnd,Prueba_Piloto_bnd,Super_rentables_bnd,Corredores_bnd,cltv_decile,Aleatorio,Segmentos_sept,segmento_deficiente_bnd,CAPTOTAL_vida,Edad_tomador_bnd,empleados,POL_VIDA,bm_anualidad,BM_cat,IsBMP,Random,Random_22,DEFENDIBLE,bm_ampliado,Stros2,CS,Provincia,m_anul,Sdadpol_12M_anterior,stros_anterior,defendida_SAS,defendida_ant,FR,priman_new_fr,POLIZAR,PAQUETE_REPORTING,paquete_reporting_reempl,polizar_post,tipo_reemplazo,FRFINAL,GARANTIA_1,GARANTIA_2,GARANTIA_3,GARANTIA_4,GARANTIA_5,GARANTIA_6,GARANTIA_7,GARANTIA_8,Gar_missing,MODALIDAD,SEX,ASISTENCIA,COND_TOM,COND_PROP,MATRICULA,CILIND,POTENCIA,CLASEVH,VALORVH,MARCA,MODELO,BASICO,ROBO,DANOS,FRANQUI,PAQUETE,PLAZAS,GRUA,NPROPIETARIO,ID_CLIENTE,FNACI8,FCARN8,FMATRI,ANTIG_VEHICULO,BNS_TIPO_CONSULTA,APR_new,TPR_new,APR_old,TPR_old,CANAL_Med,Nombre_Med,Mediador_Principal_Med,MEDIADOR_Med,solicitud_Med,coti_tec_Med,version_coti_Med,poli_Med,Reemplazo_Med,BM_ACTUAL_Med,BM_ACTUAL_pol_Med,Dto_aplicado_coti_Med,Dto_aplicado_poli_Med,tirea_NO_deseado_Med,FP_NO_Anual_Med,diferente_CP_Med,Dcto_coti_CAP_Med,Dcto_coti_VC_Med,Dcto_coti_TRA_Med,Dcto_coti_CAPVC_Med,Dcto_coti_VCTRA_Med,Dcto_poli_CAP_Med,Dcto_poli_VC_Med,Dcto_poli_TRA_Med,Dcto_poli_CAPVC_Med,Dcto_poli_VCTRA_Med,PRIANUALAC_Med,Canc_NB_Med,Canc_RW_Med,Canc_Impag_NB_Med,Canc_Impag_RW_Med,exp_Med,prima_ganada_Med,Sini_tot_Med,Coste_tot_Med,Sini_RC_Med,Coste_RC_Med,Sini_Danos_Med,Coste_Danos_Med,exp_Danos_Med,Desbloqueo_Med,Desbl_FalsoReemp_Med,Desbl_Menores_Med,DEFENDIDA_Med,DEFENDIBLE_Med,Ratio_DEFENSA_Med,Coste_defensa_Med,tot_MEDIADORES_Med,Ratio_Solicitud_Med,Ratio_Cotizacion_Med,Ratio_Reemplazo_Med,Ratio_BM_Med,Ratio_Tirea_No_deseado_Med,Ratio_Diferente_CP_Med,Ratio_Poli_CAP_Med,Ratio_Poli_VC_Med,Ratio_Poli_TRA_Med,Ratio_Canc_NB_Med,Ratio_Canc_RW_Med,Ratio_Canc_impag_NB_Med,Ratio_Canc_impag_RW_Med,Ratio_Coste_Defensa_Med,Ratio_Fracc_Med,Ratio_Desbloqueo_Med,Ratio_Desbl_FalsoReemp_Med,Ratio_Desbl_Menores_Med,desv_Solicitud_Med,desv_Cotizacion_Med,desv_Reemplazo_Med,desv_BM_Med,desv_Tirea_No_deseado_Med,desv_Diferente_CP_Med,desv_Poli_CAP_Med,desv_Poli_VC_Med,desv_Poli_TRA_Med,desv_Canc_NB_Med,desv_Canc_RW_Med,desv_Canc_impag_NB_Med,desv_Canc_impag_RW_Med,desv_Defensa_Med,desv_Coste_Defensa_Med,desv_Fracc_Med,desv_Desbloqueo_Med,desv_Desbl_FalsoReemp_Med,desv_Desbl_Menores_Med
0,23175501,0,1265,608.47,568.66,211.00,2023-01-03,1020,"608,47",None,None,None,None,0,0.00,0.00,"18,006.00",1,28062,29,519,11000,13379484,Agente,"28,062.00",-Ninguno,None,"0,00","549,10","0,00",1,1,-69.33,0,0.00,0,-23.30,-14.83,15.00,77.00,Menor 160%,NaN,0.73,0.92,0.77,374.06,0.00,0.00,0.00,N,Media,Resto,1.00,Resto,Resto,8.00,No,NaN,Resto,None,None,None,None,None,None,None,None,None,None,NaN,None,9,18,NaN,0.00,0.00,N,N,NaN,None,0,211-Todo Riesgo con Franquicia,None,0,None,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,0.00,Hombre,S,S,S,0669GBF,"1,560.00",90.00,T,"19,100.00",PEUGEOT,308 CONFORT 1.6 HDI 90 5P,S,S,S,S,4,5.00,None,"13,379,484.00","13,379,484.00",1946-07-20,1968-02-02,2008-02-01,16.00,CT,"608,47","338,95619130502109","608,47","419,3584640900923",D,GARCIA CUERVA JUAN DE DIOS,"28,062.00","28,062.00",NaN,11.00,27.00,5.00,NaN,9.22,6.60,0.10,0.15,NaN,1.00,NaN,14.00,NaN,NaN,8.00,NaN,1.00,NaN,NaN,4.00,NaN,"2,048.06",NaN,117.00,3.00,62.00,37.00,"14,509.96",13.00,"5,604.31",1.00,0.00,10.00,"2,558.03",8.96,21.00,2.00,NaN,62.00,147.00,0.42,"2,070.34","5,784.00",NaN,0.41,NaN,6.60,NaN,NaN,0.20,NaN,NaN,NaN,23.40,0.60,12.40,41

In [48]:
BBDD_VF.FRFINAL.value_counts()

FRFINAL
0.00    1841963
1.00      16363
Name: count, dtype: int64

### Guardar PQ Final con BBDD

In [49]:
BBDD_VF.to_parquet(os.path.join(file_path, "FR_Tomador.parquet"))